In [96]:
# import important modules
import numpy as np
import pandas as pd
# sklearn modules
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB # classifier 
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    plot_confusion_matrix,
)
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
# text preprocessing modules
from string import punctuation 
# text preprocessing modules
from nltk.tokenize import word_tokenize
import nltk
# adding two more downloads based on warnings generated when cleaning text 
nltk.download('stopwords')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
import re #regular expression
# Download dependency
for dependency in (
    "brown",
    "names",
    "wordnet",
    "averaged_perceptron_tagger",
    "universal_tagset",
):
    nltk.download(dependency)
    
import warnings
warnings.filterwarnings("ignore")
# seeding
np.random.seed(123)


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/emilyjiang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/emilyjiang/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package brown to
[nltk_data]     /Users/emilyjiang/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package names to
[nltk_data]     /Users/emilyjiang/nltk_data...
[nltk_data]   Package names is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/emilyjiang/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/emilyjiang/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /Users/emilyjiang/nlt

In [97]:
import pandas as pd

In [98]:
# import tensorflow and keras for padding 
import tensorflow as tf 

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/emilyjiang/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/6q/ndm97whn24zdlr8rtgm_lr1w0000gn/T/ipykernel_94573/4074912119.py", line 2, in <cell line: 2>
    import tensorflow as tf
  File "/Users/emilyjiang/opt/anaconda3/lib/python3.9/site-packages/tensorflow/__init__.py", line 37, in <module>
    from tensorflow.python.tools import module_util as _module_util
  File "/Users/emilyjiang/opt/anaconda3/lib/python3.9/site-packages/tensorflow/python/__init__.py", line 42, in <module>
    from tensorflow.python import data
  File "/Users/emilyjiang/opt/anaconda3/lib/python3.9/site-packages/tensorflow/python/data/__init__.py", line 21, in <module>
    from tensorflow.python.data import experimental
  File "/Users/emilyjiang/opt/anaconda3/lib/python3.9/site-packages/tensorflow/python/data/experimental/__init__.py", line

In [99]:
import keras as k

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/emilyjiang/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/6q/ndm97whn24zdlr8rtgm_lr1w0000gn/T/ipykernel_94573/643355293.py", line 1, in <cell line: 1>
    import keras as k
  File "/Users/emilyjiang/opt/anaconda3/lib/python3.9/site-packages/keras/__init__.py", line 21, in <module>
    from keras import models
  File "/Users/emilyjiang/opt/anaconda3/lib/python3.9/site-packages/keras/models/__init__.py", line 18, in <module>
    from keras.engine.functional import Functional
  File "/Users/emilyjiang/opt/anaconda3/lib/python3.9/site-packages/keras/engine/functional.py", line 24, in <module>
    import tensorflow.compat.v2 as tf
  File "/Users/emilyjiang/opt/anaconda3/lib/python3.9/site-packages/tensorflow/__init__.py", line 37, in <module>
    from tensorflow.python.tools import module_util as _module_util
  File

In [100]:
from keras_preprocessing.sequence import pad_sequences

In [101]:
from keras_preprocessing.text import Tokenizer

In [102]:
# load data from phyb_400 csv 
df_original = pd.read_csv("/Users/emilyjiang/Desktop/QMIND_Ciaran_Webscraping/PHYB_400.csv", index_col=0) 

# print beginning of data to check 
df_original.head()

,authors,ArticleTitle,journal_title,volume,date,pubmed,doi_pii_str,abstract
0,"B. Jiang, Y. Shi, Y. Peng, Y. Jia, Y. Yan, X. ...","""Cold-Induced CBF-PIF3 Interaction Enhances Fr...",Molecular plant,13(6),(Jun. 2020).,PUBMED: 32311530;,DOI 10.1016/j.molp.2020.04.006.,Growth inhibition and cold-acclimation strateg...
1,"Z. Mao, S. He, F. Xu, X. Wei, L. Jiang, Y. Liu...","""Photoexcited CRY1 and phyB interact directly ...",The New phytologist,225(2),(Jan. 2020).,PUBMED: 31514232;,DOI 10.1111/nph.16194.,Arabidopsis CRY1 and phyB are the primary blue...
2,"F. Zheng, Y. Wang, D. Gu and X. Liu,","""Histone Deacetylase HDA15 Restrains PHYB-Depe...",Cells,11(23),(Nov. 2022).,PUBMED: 36497048;,DOI 10.3390/cells11233788.,Seed germination is essential for the coloniza...
3,"JH. Jung, M. Domijan, C. Klose, S. Biswas, D. ...","""Phytochromes function as thermosensors in Ara...","Science (New York, N.Y.)",354(6314),(Nov. 2016).,PUBMED: 27789797;,NaN,"Plants are responsive to temperature, and some..."
4,"T. Yan, Y. Heng, W. Wang, J. Li and XW. Deng,","""SWELLMAP 2, a phyB-Interacting Splicing Facto...",Frontiers in plant science,13,(2022).,PUBMED: 35222493;,DOI 10.3389/fpls.2022.836519.,Light-triggered transcriptome reprogramming is...


In [103]:
# checking shape of dataset 
df_original.shape
# dataset has 8 columns, 400 rows 

(400, 8)

In [104]:
# check if  dataset has any missing values
df_original.isnull().sum()

# check what to do with missing values? 

authors            1
ArticleTitle       0
journal_title      0
volume             7
date               0
pubmed             0
doi_pii_str      128
abstract           5
dtype: int64

Ciaran's next code: 

In [105]:
'''# import dataset
df_orig = pd.read_csv("PHYB_400.csv", index_col=0)
# download a list of useless stop words
df_orig.iloc[:,-1] = df_orig.iloc[:,-1].astype(str)
stop_words = set(stopwords.words('english')) 
# nltk.download('stopwords')
# print(nltk.download('stopwords'))
# print(filter(lambda w: not w in s, df.iloc[1,-1]))
# stop_words
df_orig

# tokenize the words (make the paragraph into a list of strings for each word)
df = df_orig

for i in tqdm(range(len(df))):
    df.iloc[i,-1] = word_tokenize(df.iloc[i,-1])
    # df.iloc[i,-1] = "".join([word + " " for word in text_tokens if not word in stopwords.words()])
# convert to the lsit to a string
df.iloc[:,-1] = df.iloc[:,-1].astype(str)
# strip bad characters
bad_letters = ['\'', '.', ',', '[', ']', '(', ')']
for i in bad_letters:
    df.iloc[:,-1] = df.iloc[:,-1].map(lambda x: x.replace(i,''))
# remove multiple spaces
for i in range(5):  
    df.iloc[:,-1] = df.iloc[:,-1].map(lambda x: x.replace('  ',' '))
print(df.iloc[0,-1])

#Tokenize the sentences (but using keras this time)
myTokenizer = Tokenizer(num_words=1000)
myTokenizer.fit_on_texts(df.iloc[:,-1])
sequences = myTokenizer.texts_to_sequences(df.iloc[:,-1])

# Padding ( make everything the same dimension by adding zeros to the front)
padded = pad_sequences(sequences, maxlen=len(df.iloc[:,-1][3].split(" ")))

df['Count'] = df['abstract'].map(lambda x: x.count("phyB"))
# df[df['Count'] > 0]
df['Label'] = sum([df['Count'] > 3])'''

'# import dataset\ndf_orig = pd.read_csv("PHYB_400.csv", index_col=0)\n# download a list of useless stop words\ndf_orig.iloc[:,-1] = df_orig.iloc[:,-1].astype(str)\nstop_words = set(stopwords.words(\'english\')) \n# nltk.download(\'stopwords\')\n# print(nltk.download(\'stopwords\'))\n# print(filter(lambda w: not w in s, df.iloc[1,-1]))\n# stop_words\ndf_orig\n\n# tokenize the words (make the paragraph into a list of strings for each word)\ndf = df_orig\n\nfor i in tqdm(range(len(df))):\n    df.iloc[i,-1] = word_tokenize(df.iloc[i,-1])\n    # df.iloc[i,-1] = "".join([word + " " for word in text_tokens if not word in stopwords.words()])\n# convert to the lsit to a string\ndf.iloc[:,-1] = df.iloc[:,-1].astype(str)\n# strip bad characters\nbad_letters = [\'\'\', \'.\', \',\', \'[\', \']\', \'(\', \')\']\nfor i in bad_letters:\n    df.iloc[:,-1] = df.iloc[:,-1].map(lambda x: x.replace(i,\'\'))\n# remove multiple spaces\nfor i in range(5):  \n    df.iloc[:,-1] = df.iloc[:,-1].map(lambda x: x

Taking line to read abstract of article from Ciaran's model (***CHECK THAT THAT LINE READS ABSTRACT AS STRING***): 

In [106]:
df_original.iloc[:,-1] = df_original.iloc[:,-1].astype(str)
print(df_original.iloc[:,-1])

0      Growth inhibition and cold-acclimation strateg...
1      Arabidopsis CRY1 and phyB are the primary blue...
2      Seed germination is essential for the coloniza...
3      Plants are responsive to temperature, and some...
4      Light-triggered transcriptome reprogramming is...
                             ...                        
395    The phytochrome family of red/far-red (R/FR)-r...
396    Phytochromes are red (R) and far-red (FR) ligh...
397    The phytochrome (phy) family of sensory photor...
398    Light is a crucial environmental signal that c...
399    Overexpression of phytochrome B (phyB) in Arab...
Name: abstract, Length: 400, dtype: object


Using text_cleaning() function to clean dataset of text 

In [107]:
stop_words =  stopwords.words('english')
def text_cleaning(text, remove_stop_words=True, lemmatize_words=True):
    # Clean the text, with the option to remove stop_words and to lemmatize word
    # Clean the text
    text = re.sub(r"[^A-Za-z0-9]", " ", text)
    text = re.sub(r"\'s", " ", text)
    text =  re.sub(r'http\S+',' link ', text)
    text = re.sub(r'\b\d+(?:\.\d+)?\s+', '', text) # remove numbers
        
    # Remove punctuation from text
    text = ''.join([c for c in text if c not in punctuation])
    
    # Optionally, remove stop words
    if remove_stop_words:
        text = text.split()
        text = [w for w in text if not w in stop_words]
        text = " ".join(text)
    
    # Optionally, shorten words to their stems
    if lemmatize_words:
        text = text.split()
        lemmatizer = WordNetLemmatizer() 
        lemmatized_words = [lemmatizer.lemmatize(word) for word in text]
        text = " ".join(lemmatized_words)
    
    # Return a list of words
    return(text)


Now use text_cleaning() function to clean data, accessing last column of dataframe (which is the abstract)
Creates a new column for cleaned data. Using text_cleaning function from above chunk of code, we are cleaning the text in the abstract column and saving this cleaned text in a new column

In [108]:
df_original["cleaned_data"] = df_original[df_original.columns[len(df_original.columns)-1]]
df_original.head()

,authors,ArticleTitle,journal_title,volume,date,pubmed,doi_pii_str,abstract,cleaned_data
0,"B. Jiang, Y. Shi, Y. Peng, Y. Jia, Y. Yan, X. ...","""Cold-Induced CBF-PIF3 Interaction Enhances Fr...",Molecular plant,13(6),(Jun. 2020).,PUBMED: 32311530;,DOI 10.1016/j.molp.2020.04.006.,Growth inhibition and cold-acclimation strateg...,Growth inhibition and cold-acclimation strateg...
1,"Z. Mao, S. He, F. Xu, X. Wei, L. Jiang, Y. Liu...","""Photoexcited CRY1 and phyB interact directly ...",The New phytologist,225(2),(Jan. 2020).,PUBMED: 31514232;,DOI 10.1111/nph.16194.,Arabidopsis CRY1 and phyB are the primary blue...,Arabidopsis CRY1 and phyB are the primary blue...
2,"F. Zheng, Y. Wang, D. Gu and X. Liu,","""Histone Deacetylase HDA15 Restrains PHYB-Depe...",Cells,11(23),(Nov. 2022).,PUBMED: 36497048;,DOI 10.3390/cells11233788.,Seed germination is essential for the coloniza...,Seed germination is essential for the coloniza...
3,"JH. Jung, M. Domijan, C. Klose, S. Biswas, D. ...","""Phytochromes function as thermosensors in Ara...","Science (New York, N.Y.)",354(6314),(Nov. 2016).,PUBMED: 27789797;,NaN,"Plants are responsive to temperature, and some...","Plants are responsive to temperature, and some..."
4,"T. Yan, Y. Heng, W. Wang, J. Li and XW. Deng,","""SWELLMAP 2, a phyB-Interacting Splicing Facto...",Frontiers in plant science,13,(2022).,PUBMED: 35222493;,DOI 10.3389/fpls.2022.836519.,Light-triggered transcriptome reprogramming is...,Light-triggered transcriptome reprogramming is...


In [109]:
myTokenizer = Tokenizer(num_words=1000)

Tokenizing data, padding data, counting occurences of "phyB"

In [110]:
# tokenize the sentences (but using keras this time)
myTokenizer.fit_on_texts(df_original.iloc[:,-1])
sequences = myTokenizer.texts_to_sequences(df_original.iloc[:,-1])

# padding (make everything the same dimension by adding zeros to the front)
padded = pad_sequences(sequences, maxlen=len(df_original.iloc[:,-1][3].split(" ")))

# count number of "phyB" occurrences in cleaned text column 
df_original['count'] = df_original[df_original.columns[len(df_original.columns)-1]].map(lambda x: x.count("phyB"))
# df_original[df_original['count'] > 0]
df_original['label'] = sum([df_original['count'] > 3])

In [111]:
print(df_original['label']) 
sum(df_original['label'])

0      1
1      1
2      0
3      0
4      1
      ..
395    0
396    0
397    1
398    0
399    1
Name: label, Length: 400, dtype: int64


189

In [112]:
x = df_original["cleaned_data"] 
y = df_original.label.values 

Split dataset into train and test data, test = 15% of dataset 

In [113]:
x_train, x_valid, y_train, y_valid = train_test_split(x, y, 
test_size=0.30,
random_state=42, 
shuffle=True,
stratify=y,
)

Creating NLP Model: Now training Multinomial Naive Bayes algorithm (common algorithm for text classification) to classify if article is about phyB or not

Transform cleaned data into numerical values so model can understand it. Using TfidfVectorizer method from scikit-learn (converts text documents to matrix of TF-IDF features)

Use Pipeline class from scikit-learn to apply list of transforms and final estimator for pre-processing and training 

In [114]:
phyB_classifier = Pipeline(steps=[
    ('pre_processing',TfidfVectorizer(lowercase=False)),
    ('naive_bayes',MultinomialNB())
])

Training classifier

In [115]:
phyB_classifier.fit(x_train, y_train) 

Pipeline(steps=[('pre_processing', TfidfVectorizer(lowercase=False)),
                ('naive_bayes', MultinomialNB())])

Making predictiom from validation set 

In [116]:
y_predict = phyB_classifier.predict(x_valid) 

Scoring accuracy

In [117]:
accuracy_score(y_valid, y_predict) 

0.6833333333333333

Save model pipeline 

In [118]:
import joblib
# import it to the webscraping folder
joblib.dump(phyB_classifier, '/Users/emilyjiang/Desktop/Webscraping')

['/Users/emilyjiang/Desktop/Webscraping']